# 데이터 가져오기

In [1]:
import os
import json
import glob
from soynlp.noun import LRNounExtractor

# 📌 JSON 파일이 들어있는 폴더 경로 (사용자의 폴더 경로로 수정)
folder_path = "../../../게임"

# 📌 폴더 내 모든 JSON 파일 불러오기 (CODE0001.json, CODE0002.json, ...)
json_files = glob.glob(os.path.join(folder_path, "BO*.json"))

# 📌 모든 파일에서 댓글(content)만 추출
comments = []
for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)  # JSON 파일 로드

        # 'text' 항목에서 'content'만 추출
        for entry in data["SJML"]["text"]:
            comments.append(entry["content"])

print(f"📌 총 {len(comments)}개의 댓글을 수집했습니다!")




📌 총 840212개의 댓글을 수집했습니다!


In [2]:
import re
# 📌 괄호와 괄호 안의 내용을 제거하는 함수
def remove_parentheses(text):
    return re.sub(r'\([^)]*\)', '', text)

# 📌 모든 댓글에 대해 괄호와 괄호 안의 내용을 제거
comments = [remove_parentheses(comment) for comment in comments]

print(f"📌 전처리 후 {len(comments)}개의 댓글을 수집했습니다!")

comments[:5]

📌 전처리 후 840212개의 댓글을 수집했습니다!


['7빠 야효', '양경철 양경철 hello', '구름빵 how sit goi bff', '유림 hi', '1빠다 따석들아']

# 텍스트 분할

In [4]:
import random

# 📌 데이터 셔플링 (무작위 순서로 섞기)
random.shuffle(comments)

# 📌 50:50 데이터 분할 (42만 개씩)
split_index = len(comments) // 2
train_data = comments[:30000]  # 학습 데이터 42만 개
test_data = comments[split_index:]  # 테스트 데이터 42만 개

# 📌 데이터 개수 확인
print(f"학습 데이터 개수: {len(train_data)}")
print(f"테스트 데이터 개수: {len(test_data)}")

학습 데이터 개수: 30000
테스트 데이터 개수: 420106


# Train

In [5]:
import json

# 📌 nounextractor 학습
noun_extractor = LRNounExtractor()
nouns = noun_extractor.train_extract(
    train_data,
    min_noun_score=0.3,
    min_noun_frequency=20
)



[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (2402, 1459) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 573 nouns are extracted


In [7]:
# Extract 10 nouns from the trained model
top_10_nouns = list(nouns.items())[:10]
print(top_10_nouns)

[('때문', NounScore_v1(frequency=78, score=0.9997221142857142, known_r_ratio=0.9333333333333333)), ('도움', NounScore_v1(frequency=18, score=0.968345625, known_r_ratio=1.0)), ('이랑', NounScore_v1(frequency=274, score=0.7998286, known_r_ratio=0.7142857142857143)), ('국민', NounScore_v1(frequency=6, score=0.8774915135135136, known_r_ratio=0.8705882352941177)), ('내가', NounScore_v1(frequency=173, score=0.9913243333333334, known_r_ratio=0.6)), ('닉네', NounScore_v1(frequency=32, score=0.999185, known_r_ratio=0.03125)), ('자주', NounScore_v1(frequency=42, score=0.9887579999999999, known_r_ratio=0.75)), ('떡국', NounScore_v1(frequency=22, score=0.6561567777777778, known_r_ratio=0.9)), ('뭐라', NounScore_v1(frequency=14, score=0.48428875, known_r_ratio=1.0)), ('컨셉', NounScore_v1(frequency=32, score=0.9103542222222223, known_r_ratio=0.72))]


In [8]:
top100 = sorted(nouns.items(), 
    key=lambda x:-x[1].frequency * x[1].score)[:100]

for i, (word, score) in enumerate(top100):
    if i % 5 == 0:
        print()
    print('%6s (%.2f)' % (word, score.score), end='')


    꿀잼 (0.99)    너무 (0.57)    사랑 (0.97)    진짜 (0.70)    제가 (0.99)
   좋아요 (0.54)    감사 (0.94)    오늘 (0.33)   만약에 (0.66)    정말 (0.81)
    다음 (1.00)   목소리 (0.73)    축하 (0.77)    재미 (0.48)    처음 (1.00)
    이랑 (0.80)   오랜만 (0.96)    좋아 (0.61)    생일 (0.76)   코아님 (0.51)
    보고 (0.42)    완전 (1.00)    마크 (0.77)    안녕 (1.00)    내가 (0.99)
   어떻게 (0.85)    게임 (0.61)    노래 (0.53)   마지막 (0.93)   이렇게 (1.00)
    댓글 (0.63)   친구들 (0.94)    하트 (0.87)  샌드박스 (0.70)    제일 (1.00)
    대박 (0.86)    나이 (0.81)   구독자 (0.76)    응원 (0.99)    생각 (0.75)
   유튜브 (0.66)    스킨 (0.66)    하지 (1.00)    인정 (0.83)    죄송 (0.94)
    잠뜰 (0.51)   화이팅 (0.69)    엄마 (0.83)    하고 (0.50)    같이 (0.80)
   있는데 (1.00)    한번 (0.69)    엄청 (0.62)  로블록스 (0.65)    학교 (0.83)
   오프닝 (0.58)   봤는데 (1.00)    시간 (0.80)   꾸르잼 (1.00)    사람 (0.44)
    넘날 (0.75)   32분 (1.00)    친구 (0.69)    부탁 (0.96)   도도한 (0.94)
    보니 (1.00)    이번 (0.90)    이름 (0.38)    머리 (0.97)   열심히 (1.00)
    내일 (0.63)    생신 (0.82)    고정 (0.60)   갑자기 (1.00)    점프 (0.91)
    때문 (1